Here’s the list of reviews of Chrome apps - scraped from Playstore.  DataSet Link

Problem statement - There are times when a user writes Good, Nice App or any other positive text, in the review and gives 1-star rating. Your goal is to identify the reviews where the semantics of review text does not match rating. 

Your goal is to identify such ratings where review text is good, but rating is negative- so that the support team can point this to users. 


In [1]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import flair
from flair.data import Sentence
from flair.models import TextClassifier
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
from textblob import TextBlob

In [5]:
data = pd.read_csv('chrome_reviews.csv')
data.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [6]:
data.info

<bound method DataFrame.info of           ID                                         Review URL  \
0       3886  https://play.google.com/store/apps/details?id=...   
1       3887  https://play.google.com/store/apps/details?id=...   
2       3888  https://play.google.com/store/apps/details?id=...   
3       3889  https://play.google.com/store/apps/details?id=...   
4       3890  https://play.google.com/store/apps/details?id=...   
...      ...                                                ...   
7199  684987  https://play.google.com/store/apps/details?id=...   
7200  684988  https://play.google.com/store/apps/details?id=...   
7201  684989  https://play.google.com/store/apps/details?id=...   
7202  684990  https://play.google.com/store/apps/details?id=...   
7203  684991  https://play.google.com/store/apps/details?id=...   

                                                   Text  Star  Thumbs Up  \
0                            This is very helpfull aap.     5          0   
1          

In [7]:
data.shape

(7204, 10)

In [8]:
data.isnull().sum()

ID                    0
Review URL            0
Text                  1
Star                  0
Thumbs Up             0
User Name             0
Developer Reply    7109
Version              85
Review Date           0
App ID                0
dtype: int64

In [9]:
## from that we take important future
data = data[["Text","Star","Thumbs Up"]]

In [10]:
data[data['Text'].isnull()]

,Text,Star,Thumbs Up
5163,NaN,5,0


In [11]:
data= data.dropna()

In [12]:
data.isnull().sum()

Text         0
Star         0
Thumbs Up    0
dtype: int64

In [13]:
sd = SentimentIntensityAnalyzer()

In [14]:
data["compound_score"]= [sd.polarity_scores(text)["compound"]for text in data["Text"].values]

In [15]:
data.head()

,Text,Star,Thumbs Up,compound_score
0,This is very helpfull aap.,5,0,0.0000
1,Good,3,2,0.4404
2,Not able to update. Neither able to uninstall.,1,0,0.0000
3,Nice app,4,0,0.4215
4,Many unwanted ads,1,0,-0.2263


In [16]:
data["vader_score"]=[1 if x>0.05 else 0 for x in data['compound_score']]

In [17]:
flr =TextClassifier.load('en-sentiment')

2022-06-26 16:10:54,273 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [18]:
sentiment =[]
connfidence=[]
for sentence in data['Text']:
    sentence = Sentence(sentence)
    flr.predict(sentence)
    sentiment.append(sentence.labels[0].value)
    connfidence.append(sentence.labels[0].score)


In [19]:
data['sentiment']=sentiment
data['connfidence']=connfidence

In [21]:
data['flair_score']= [1 if x=='POSITIVE'else 0 for x in data['sentiment']]

In [33]:
data

,Text,Star,Thumbs Up,compound_score,vader_score,sentiment,connfidence,flair_score
0,This is very helpfull aap.,5,0,0.0000,0,POSITIVE,0.979062,1
1,Good,3,2,0.4404,1,POSITIVE,0.931395,1
2,Not able to update. Neither able to uninstall.,1,0,0.0000,0,NEGATIVE,0.999945,0
3,Nice app,4,0,0.4215,1,POSITIVE,0.970572,1
4,Many unwanted ads,1,0,-0.2263,0,NEGATIVE,0.999877,0
...,...,...,...,...,...,...,...,...
7199,Bagusss..,5,0,0.0000,0,POSITIVE,0.965633,1
7200,Bad version 😔,1,0,-0.5423,0,NEGATIVE,0.999728,0
7201,One thing that I have to say I can't spelled t...,5,0,0.8395,1,POSITIVE,0.997463,1
7202,Excellent,5,0,0.5719,1,POSITIVE,0.985055,1
